In [ ]:
import pandas as pd
import pandas as pd
from pykeen.datasets import PathDataset
from pykeen.pipeline import pipeline
from neo4j import GraphDatabase
from pykeen.triples import TriplesFactory
import numpy as np
from sklearn.model_selection import train_test_split

uri = "bolt://localhost:7687"  # Update with your Neo4j URI
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))  # Update with your credentials

uri = "bolt://localhost:7687"  # Update with your Neo4j URI
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))  # Update with your credentials



def fetch_triples():
    query = """
  MATCH (rev:Review)-[:indicates_perceived_cleaning_quality]->(ca:Quality_Indication)
    RETURN rev.text AS Review_Text, 
           "indicates_perceived_cleaning_quality",
           ca.text as  Quality_Indication
    """
    with driver.session() as session:
        result = session.run(query)
        return [(record["Review_Text"], "indicates_perceived_cleaning_quality", record["Quality_Indication"]) for
                record in result]
    
    
triple = fetch_triples()
df = pd.DataFrame(triple, columns=['Review_Text', 'indicates_perceived_cleaning_quality','Quality_Indication'])

df

In [ ]:
from pykeen.triples import TriplesFactory

# Assuming your dataframe has columns 'subject', 'predicate', 'object'
triples_factory = TriplesFactory.from_labeled_triples(
    triples=df[['Review_Text', 'indicates_perceived_cleaning_quality', 'Quality_Indication']].values,
)

training = triples_factory
validation = triples_factory
testing = triples_factory

d=training
id_to_entity={v: k for k, v in d.entity_to_id.items()}
id_to_relation={v: k for k, v in d.relation_to_id.items()}

# Display the first few triples
triples_factory.triples

In [ ]:
from pykeen.pipeline import pipeline



result = pipeline(
    model='TransE',
    loss="softplus",
    training=training,
    testing=testing,
    validation=validation,
    model_kwargs=dict(embedding_dim=3),  # Increase the embedding dimension
    optimizer_kwargs=dict(lr=0.1),  # Adjust the learning rate
    training_kwargs=dict(num_epochs=100, use_tqdm_batch=False),  # Increase the number of epochs
)

# The trained model is stored in the pipeline result
model = result.model

In [ ]:
from pykeen.evaluation import RankBasedEvaluator

# Create an evaluator
evaluator = RankBasedEvaluator()

# Evaluate the model
metrics = evaluator.evaluate(result.model, testing.mapped_triples, additional_filter_triples=[training.mapped_triples, validation.mapped_triples])

# Print the metrics
print(f"Hits@1: {metrics.get_metric('hits@1')}")
print(f"Hits@3: {metrics.get_metric('hits@3')}")
print(f"Hits@5: {metrics.get_metric('hits@5')}")
print(f"Hits@10: {metrics.get_metric('hits@10')}")
print(f"Mean Reciprocal Rank: {metrics.get_metric('mean_reciprocal_rank')}")

In [ ]:
from pykeen.predict import predict_triples
dataset = get_dataset(dataset="nations")
pack = predict_triples(model=result.model, triples=dataset.validation) #Hier muss ich einfach alle Möglichkeiten angeben und bekomme scores

In [ ]:
def fetch_reviews_without_quality_connection():
    pre_predictions = {"reviews": {}, "cleaners": {}}

    query = """
    // Match all reviews and cleaners
    MATCH (rev:Review), (r:Reinigungsmitarbeiter)

     // Use OPTIONAL MATCH to find existing connections
     OPTIONAL MATCH (rev)-[:HAS_PERCEIVED_CLEANING]->(ca:PerceivedCleaningQuality)-[:INDICATES_CLEANING_QUALITY]->(r)

     // Filter out reviews and cleaners that have these connections
     WHERE ca IS NULL

      RETURN rev.text AS Review_Text, 
             r.name AS ReinigungsMitarbeiter
    """
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            review_text = record["Review_Text"]
            cleaner_name = record["ReinigungsMitarbeiter"]

            # Add to predictions dictionary
            if review_text not in pre_predictions["reviews"]:
                pre_predictions["reviews"][review_text] = []
            if cleaner_name not in pre_predictions["cleaners"]:
                pre_predictions["cleaners"][cleaner_name] = []

            # Add cleaner to the review's list and vice versa
            pre_predictions["reviews"][review_text].append(cleaner_name)
            pre_predictions["cleaners"][cleaner_name].append(review_text)

    # Close the driver connection
    driver.close()

    return pre_predictions
fetch_reviews_without_quality_connection()

In [ ]:
import pandas as pd
from itertools import product

reviews_wo_relation = fetch_reviews_without_quality_connection()

df = pd.DataFrame(reviews_wo_relation, columns=['Review_Text'])

# List of relations
relations = ['A', 'B', 'C']

# Generate all combinations of keys with relations
combinations = list(product(df['Review_Text'], relations))

# Convert combinations to a DataFrame
df_combinations = pd.DataFrame(combinations, columns=['key', 'relation'])

print(df_combinations)